@copy right: Ryan Amer

## General Setup

In [6]:
from __future__ import division
from IPython.display import clear_output
# imports to run OpenAI Gym in Jupyter
import gym
import matplotlib.pyplot as plt
from IPython import display
# import to do training
from tpg.tpg_trainer import TpgTrainer
# import to run an agent (always needed)
from tpg.tpg_agent import TpgAgent

# how to render in Jupyter: 
# https://stackoverflow.com/questions/40195740/how-to-run-openai-gym-render-over-a-server
# https://www.youtube.com/watch?v=O84KgRt6AJI
def show_state(env, step=0, name='', info=''):
    plt.figure()
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % (name, step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())
    
# transforms the state into what the tpg agent can use.
# From 3D to 1D, taking only red data (from rgb array)
def getState(state):
    state2 = []
    for x in state:
        for y in x:
            state2.append(y[0])
            
    return state2

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
import math

init_notebook_mode(connected=True)
import multiprocessing as mp
import time


# run agent in function to work with multiprocessing
def runAgent(agenteqsq):
    agent = agenteqsq[0] # get agent
    eq = agenteqsq[1] # get environment queue
    sq = agenteqsq[2] # get score queue
    
    # check if agent already has score
    if agent.taskDone():
        sq.put((agent.getUid(), agent.getOutcomes()))
        return
        
    env = eq.get() # get an environment
    state = env.reset() # get initial state and prep environment
    score = 0
    for i in range(agenteqsq[3]): # run episodes that last 200 frames
        act = agent.act(getState(state)) # get action from agent

        # feedback from env
        state, reward, isDone, debug = env.step(act)
        score += reward # accumulate reward in score
        if isDone:
            break # end early if losing state
            
    lock.acquire() # may not actually need, mp is weird in python
    agent.reward(score) # must reward agent
    lock.release()
    
    sq.put((agent.getUid(), agent.getOutcomes())) # get outcomes with id
    eq.put(env) # put environment back
    
def gamebegin(generation,gametitle,processes,frames, rs, tpopsize, rtpopSize,
            curr_gap, pLearnerD, pLearnerA, pMutation,
            pAIT, mts, mps,
            ppd, ppa, pps,
            ppm, ppit, tgap,
            ar):
    
    action = env.action_space.n # get number of game action space

    tStart = time.time()

    lock = mp.Lock()

    trainer = TpgTrainer(actions=action, randSeed=rs, teamPopSize=tpopsize, rTeamPopSize=rtpopSize,
                    gap=curr_gap, pLearnerDelete=pLearnerD, pLearnerAdd=pLearnerA, pMutateAction=pMutation,
                    pActionIsTeam=pAIT, maxTeamSize=mts, maxProgramSize=mps,
                    pProgramDelete=ppd, pProgramAdd=ppa, pProgramSwap=pps,
                    pProgramMutate=ppm, popInit=ppit, tourneyGap=tgap,
                    actionRange=ar)


    m = mp.Manager()
    envQueue = m.Queue()
    # each process needs its own environment
    for i in range(processes):
        envQueue.put(gym.make(gametitle))

    pool = mp.Pool(processes=processes)

    summaryScores = [] # record score summaries for each gen (min, max, avg)


    for gen in range(generation): # generation loop
        scoreQueue = m.Queue() # hold agents when finish, to actually apply score

        # run generation
        # skipTasks=[] so we get all agents, even if already scored,
        # just to report the obtained score for all agents.
        pool.map(runAgent, 
                     [(agent, envQueue, scoreQueue,frames) 
                      for agent in trainer.getAllAgents(skipTasks=[])])

        scores = [] # convert scores into list
        while not scoreQueue.empty():
            scores.append(scoreQueue.get())

        # apply scores
        trainer.applyScores(scores)
        trainer.evolve(tasks=[]) # go into next gen

        # at end of generation, make summary of scores
        summaryScores.append((trainer.scoreStats['min'], 
                        trainer.scoreStats['max'],
                        trainer.scoreStats['average'])) # min, max, avg
        print(summaryScores[len(summaryScores)-1])
    return summaryScores

In [ ]:

"""
test 1
- generation , 20 loops
"""
generation = 20 # number of generations
gametitle = 'Assault-v0'
processes = 6 # how many to run concurrently (4 is best for my local desktop)
frames = 100 #total frames each play
tpopsize = 100 #teamPopSize
rtpopSize = 0 #rTeamPopSize
rs = 0 #randseed
curr_gap = 0.5 #gap
tgap = 0.5 #tourneyGap
pLearnerD = 0.7 #learner delete
pLearnerA = 0.7 #learner add
pMutation = 0.2 #player mutation rate
pAIT = 0.5 #pActionIsTeam
mts = 5
#maxTeamSize
mps = 96 #maxProgramSize
ppd = 0.5 #pProgramDelete
ppa = 0.5 #pProgramAdd
pps = 1.0 #pProgramSwap
ppm = 1.0 #pProgramMutate
ppit = None #popInit
ar = (0.0, 1.0, 0.05) #actionRange


summary = gamebegin(generation,gametitle,processes,frames, rs, tpopsize, rtpopSize,
            curr_gap, pLearnerD, pLearnerA, pMutation,
            pAIT, mts, mps,
            ppd, ppa, pps,
            ppm, ppit, tgap,
            ar)
#legend
trace1 = go.Scatter(x=[i for i in range(len(summary))],y=[i[0] for i in summary],name='min')
trace2 = go.Scatter(x=[i for i in range(len(summary))],y=[i[1] for i in summary],name='max')
trace3 = go.Scatter(x=[i for i in range(len(summary))],y=[i[2] for i in summary],name='average')
data = [trace1,trace2,trace3]
layout = go.Layout(showlegend=True,legend={'x':0.2,'y':0.6})
fig = go.Figure(data=data,layout=layout)
iplot(fig)